# String Searching 

## Aho-Corasick Algorithm

* a fast string-searching algorithm

* depends on a fast traversal of a data structure known as the Aho-Corasick Automaton, which is essentially a Multiway Trie with some additional edges.

The Aho-Corasick Automaton takes advantage of the fact that, at any point during the trie traversal, the suffix of the path from the root to the current node could potentially also exist as a path starting at the root.

To demonstrate the construction of an Aho-Corasick Automaton, we will build one from the following database of words: A, AG, GAG, GC, GCA, C, and CAA.

First, we must build a Multiway Trie, which would be the following:

<center><img src="https://ucarecdn.com/db02678c-7aa7-4c53-b7c6-fe874a698066/" width=400/></center>

Next, we want to avoid having to restart our Multiway Trie traversal for each start position of our query sequence in order to avoid duplicate operations.

For any given node u, if a suffix of the path from the root to u exists as the prefix of some other path starting at the root, we want to somehow keep track of that connection to avoid searching the same substring again. 

For example, in the following Multiway Trie, the last 2 edges in the path from the root to the purple node (shown in red) are exactly the same as the entire path from the root to the green node (also shown in red). This means that, if we traverse to the purple node and then fail (e.g. if we scan a query sequence GCAA), instead of restarting back at the root, we could simply continue where we left off in the query, but at the green node.

<center><img src="https://ucarecdn.com/1fef7670-a01c-4fc1-ac40-29fbc7837fa9/" width=400/></center>

We can take advantage of this phenomenon by adding what are called **failure links**: edges that can be traversed if we try and fail to traverse down a non-existent child edge.

Formally, for every node u, we can create a failure link from u to a node v such that the path from the root to node v is equivalent to the longest suffix of the path from the root to node u. 

Note that we do not include the full path from the root to u as a valid suffix (otherwise the failure link would always just point to u), and we include the 0-edge path as a valid suffix (meaning, in the worst case, u will have a failure link to the root). We can use the following elegant algorithm to construct all failure links:

```
for each node 'curr' in a BFS starting at the root:
    if curr is the root or is a child of the root:
        create failure link from curr to the root
    else:
        p = parent of curr
        c = label of edge going into curr
        x = node pointed to by p's failure link
        repeat infinitely:
            if x has a child with edge label c:
                create failure link from curr to that child of x
                break
            else if x is the root:
                create failure link from curr to the root
                break
            else:
                x = node pointed to by x's failure link
```

Below is the same Multiway Trie as before, but with the failure links added as dashed red edges:

<center><img src="https://ucarecdn.com/10dac098-0d4b-471b-a065-c9fd099c5afe/" width=500/></center>

We can now linearly scan our query string, and if we ever try to go down a child edge that doesn't exist, we can simply follow the node's failure link and try again.

However, if we scan the string GCAA, we would find the words GC, GCA, and CAA, but we would fail to realize that the words C and A appears in our database. Specifically, our traversal would be as follows (shown by green edges):

<center><img src="https://ucarecdn.com/ae59d963-afbc-4595-944f-a0d7d5816bfb/" width=500/></center>

In other words, our current data structure may miss instances of words that appear as substrings of other words in our database. 

How can we fix it? We can add what are called **dictionary links**: for each node u, draw a link to the first word node you would encounter if you were to repeatedly traverse failure links. If no such word node exists, u will not have a dictionary link.

Below is the same data structure as before, but with dictionary links added as dotted purple edges:

<center><img src="https://ucarecdn.com/4659027e-7a3b-4cf3-b5a9-47e31ee0e316/" width=500/></center>

The resulting data structure is **Aho-Corasick Automaton**! 

Although this data structure is seemingly difficult to build, it is fairly simple to scan a long query sequence:

```
scan(query):
    curr = root
    for each letter c in query:
        while curr does not have a child edge labeled by c:
            if curr == root:
                go to the next iteration of the for-loop (c = next letter in query)
            else:
                curr = follow failure link from curr
        curr = follow curr's child edge labeled by c
        if curr has a dictionary link:
            output all words represented by nodes in the chain of dictionary links
        if curr is a word node:
            output curr's word
```

* O(n) worst case time complexity

## Suffix Arrays 

* $ O(n^2) $ - worst case space complexity

Let's formulate this biological question as a computational problem. We have a long database sequence D of length n (the genome), and we have a query Q of m strings, each of length k (the reads). We can assume that n >> k, and we can also assume that our database does not change frequently, but we can have many different queries. Our computational problem is the following: For every string w in Q, does w appear as a substring of D, and if so, at what position(s) of D?

As you may have noticed, with respect to the problem we solved using an Aho-Corasick Automaton, the roles of our inputs are reversed:

* Previously, our collection of short sequences was our database, and our single long sequence was our query
* Now, our collection of short sequences is our query, and our single long sequence is our database


Let's imagine an example in which our database is the string GCATCGC. 

If we were to query the string ACC, we would see that doesn't appear in the database.

If we were to query the string ATC, we would see that it appears once in the database (GCATCGC). 

If we were to query the string GC, we would see that it appears twice in the database (GCATCGC).

If you think about it, a given word w appears in our database D if and only if there exists a suffix of D that begins with w. We can highlight the suffixes from the example above as follows (bold = match of query word, and underline = corresponding suffix):

GC**ATC**GC

**GC**ATCGC

GCATC**GC**

We can use this realization to reformulate our computational problem: For every string w in Q, which suffixes of D begin with w (if any)?

* Suffix Array = a simple data structure that allows rapid substring searches of a long database string.


The idea behind the Suffix Array is as follows: 
* sort all (non-empty) suffixes of the database string in ascending alphabetical order
* we will be able to search for query strings via binary search

For example, the string GCATCGC has the following suffixes (the number to the left indicates the 0-based starting index of the suffix):

* 0: GCATCGC → GCATCGC
* 1: GCATCGC → CATCGC
* 2: GCATCGC → ATCGC
* 3: GCATCGC → TCGC
* 4: GCATCGC → CGC
* 5: GCATCGC → GC
* 6: GCATCGC → C

If we sort the suffixes alphabetically, we would get the following list of strings:
 
* 2: ATCGC
* 6: C
* 1: CATCGC
* 4: CGC
* 5: GC
* 0: GCATCGC
* 3: TCGC

Now, given a query string w, I can simply perform a binary search on this sorted list of suffixes to determine if w exists in original string! 

Specifically, in binary search, "success" is defined as finding a suffix that begins with w (i.e., I would only compare w against the first |w| characters of any given suffix).

Notice that, if I store the database string D in memory, I can represent each suffix using a single integer: the index of D at which the suffix begins.

Given D and a suffix index, I can reconstruct the suffix on-the-fly as needed.

Therefore, I can save a ton of space by only storing D and the suffix indices.

In our prior example, with a database string D = GCATCGC, we had the following (unsorted) suffixes:

* 0: GCATCGC → GCATCGC
* 1: GCATCGC → CATCGC
* 2: GCATCGC → ATCGC
* 3: GCATCGC → TCGC
* 4: GCATCGC → CGC
* 5: GCATCGC → GC
* 6: GCATCGC → C

Given that we have D in memory, we can instead represent our (unsorted) suffixes as the following array:

$$ [0, 1, 2, 3, 4, 5, 6]$$

We can then sort this array of integers using a custom comparator where, when we want to compare numbers i and j, we will return the result of comparing the suffixes of D starting at indices i and j, respectively. In this specific example, this custom sort will yield the following array:

$$[2, 6, 1, 4, 5, 0, 3]$$

This sorted array of suffix indices is precisely our Suffix Array!

* worst-case time complexity of the best sorting algorithms - O(n log n)

> However, for each of our comparisons, we may end up comparing O(n) characters, as we're comparing the suffixes defined by the indices, not the indices themselves. Thus, this simple algorithm to build a Suffix Array has a O(n² log n) time complexity in the worst case.

In 2016, Zhizhe Li, Jian Li, and Hongwei Huo introduced the first Suffix Array construction algorithm that has a O(n) time complexity in the worst-case with O(1) memory overhead to perform the construction. Note that the overall memory complexity is still O(n), as you need to store O(n) indices: the memory overhead is O(1).

To determine whether or not a given query string w exists in D: 

* simply execute the binary search algorithm on the sorted suffixes
* if we ever find a suffix that begins with w, success! 
* If our binary search algorithm terminates without every finding such a scenario, fail!

For example, to search for the query w = GC in our database string D = GCATCGC, we would have the following: 

<center><img src="https://ucarecdn.com/057ed229-8f73-4b8a-b35d-db72e8448c6c/" width=500/></center>

However, remember that we're not only interested in determining whether or not w exists in D: we actually want to find all positions at which w occurs in D! 

Notice that, because we have sorted the suffixes of D, all suffixes starting with w clump together in the Suffix Array. 

Therefore, if we can find the first and last suffixes starting with w, we would be guaranteed that all occurrences of w occur in the range between those two suffixes!

<center><img src="https://ucarecdn.com/a7684d4a-447d-4d1a-98fb-0c255a2ff3cd/" width=200/></center>

It turns out that, in order to find all occurrences of our query word w (which is equivalent to finding the first and last suffix beginning with w), we can make a tiny adjustment to our search algorithm. 

Specifically, instead of doing a single binary search to determine whether or not we have any matches, do two binary searches with slightly different "success" criteria:

* Find the i-th suffix that begins with w such that the (i-1)-th suffix does not start with w. This is the start of the range of matches
* Find the j-th suffix that begins with w such that the (j+1)-th suffix does not start with w. This is the end of the range of matches

> Note that, if the first binary search (to find the start of the range of matches) fails, we can skip the second binary search entirely, as w is guaranteed to not exist in D. 

> Also note that, when we do the second binary search (to find the end of the range of matches), we don't have to start our binary search with the full range of the Suffix Array: instead, because we're guaranteed that the j-th suffix must come after the i-th suffix, we can perform our second binary search in the range between i and the end of the Suffix Array.

For example, to search for the query w = GC in our database string D = GCATCGC, we would have the following first binary search to find the start of the range of matches. Notice how, even though we we found a match to GC in the second part of the figure (GCATCGC), it was not yet a success: the suffix before our match also started with GC (GC), which we cannot have. We specifically need to find a suffix beginning with GC such that the suffix before it does not begin with GC, which we find in the third part of the figure (GC, which is preceded by CGC, which does not start with GC).

<center><img src="https://ucarecdn.com/4babfa13-19d8-4b3d-9237-631c2c0155da/" width=600/></center>

Now that we've found the start of our range (suffix 5: GC), we can perform our second binary search to find the end of the range of matches. Remember: we need to find a suffix that begins with GC and that is followed by a suffix that does not begin with GC.

<center><img src="https://ucarecdn.com/c92df074-b92f-46d0-9ff1-ef55825552d1/" width=400/></center>

Now that we've found the end of our range (suffix 0: GCATCGC), we're done! We can output all suffix indices from start to end, which in this case is just indices 5 and 0.

Given a database string D of length n, we can find all occurrences of a query string w of length k in O(k log n) time in the worst-case (binary search with potentially k letters of comparison), and if our query Q is a collection of m strings (each of length k), we can find all occurrences of all of the m query strings in O(mk log n). 

Importantly, each of those O(m) binary search operations can be done completely independently, meaning this algorithm can be massively-parallelized if m is large. We can also construct the Suffix Array itself in O(n), though we only have to do that once overall regardless of how many query sequences we want to find.

## Burrows-Wheeler Transform

* find query string w of length k = O(k) worst-time complexity 
* find query Q is a collection of m strings (each of length k) = O(mk) 

In the world of data compression, there is a form of lossless data compression known as run-length encoding (RLE), in which runs of data (sequences in which the same value occurs multiple times consecutively) are stored as a single data value and a count.

For example, consider the string NNNNNNNNNNNNNMA. 

We can use RLE to encode it as N13MA, which is much more compact. 

As you might imagine, RLE works best when identical characters are clumped together, which poses the following question: Given some string s, can we somehow transform s such that identical characters tend to clump together, but such that we can reconstruct s from the transformation? If we are able to find such a transformation, we could potentially improve data compression:

* To compress a file, first transform it and then compress the transformation
* To decompress a file, decompress the compressed transformation, and then reconstruct the original file

In 1994, Michael Burrows and David Wheeler invented a transformation, now known as the Burrows-Wheeler Transform (BWT), that rearranges the characters of a strings such that similar characters tend to occur in runs, but that is reversible (meaning we can reconstruct the original string from its BWT).

Example: string **BANANA**

* To construct the BWT, we first need to add an end-of-string symbol that does not exist in our alphabet to the end of the string. 

> In this example, let's assume that our alphabet is the capital letters of the English alphabet (A-Z), and let's use the symbol $ (which does not occur in our alphabet) as our end-of-string symbol. Thus, our string is BANANA$.

* First, we must construct the "Burrows-Wheeler Matrix," which is simply all rotations of our string (including the end-of-string symbol):

BANANA$

$BANANA

A$BANAN

NA$BANA

ANA$BAN

NANA$BA

ANANA$B

* Then, we must sort the rotations in our Burrows-Wheeler Matrix in ascending alphabetical order (we assume our end-of-string symbol is the smallest character in alphabetical order):

$BANAN**A**

A$BANA**N**

ANA$BA**N**

ANANA$**B**

BANANA**$**

NA$BAN**A**

NANA$B**A**

The last column of the sorted Burrows-Wheeler Matrix (emphasized in bold) is our **Burrows-Wheeler Transform**! Therefore, the BWT of the string `BANANA$` is `ANNB$AA`.

As you can see, we have indeed clumped together similar symbols, which will provide great benefit to data compression.

To save memory, we could represent each rotation as a single integer: a starting index.

Then, given a starting index, we can produce the actual rotation on-the-fly by treating the original string as a circular string. Thus, this process can be done with **O(n)** memory.

However, to sort the rotations, we need to sort n strings of length n, meaning each string comparison in our sort may need to perform O(n) letter comparisons, giving this simple algorithm a O(n² log n) time complexity in the worst case. We only have to do this preprocessing once, as the database is fixed, but it can still be quite slow. However, even this can be sped up: using more sophisticated algorithms, the BWT of a string of length n can be constructed in O(n) or even faster in the worst case!

### How can we reconstruct a string from its BWT?

Given just the BWT, we absolutely know the last column of the Burrows-Wheeler Matrix (because the BWT is the last column of the Burrows-Wheeler Matrix). For example, given the BWT ANNB$AA, we can start with the following (where * denotes something we don't know yet):

`******A`

`******N`

`******N`

`******B`

`******$`

`******A`

`******A`

What else can we fill in?

Well, remember that we constructed the Burrows-Wheeler Matrix by sorting all rotations of our original string. 

Therefore, the first column simply contains the letters of our original string sorted in alphabetical order. Our BWT contains the exact same letters as our original string (just rearranged), meaning if we sort the letters of our BWT, we will get the same exact sequence of letters as we would if we sorted our original string. Therefore, the first column is simply the result of sorting our BWT alphabetically:

`$*****A`

`A*****N`

`A*****N`

`A*****B`

`B*****$`

`N*****A`

`N*****A`

It turns out that, for each letter x, the i-th occurrence of x in the first column happens to correspond to the exact same letter in the original string as the i-th occurrence of x in the last column. 

For example, in the same Burrows-Wheeler Matrix we've been working with (reproduced below), the first occurrence of A in the last column (labeled A₁) is the exact same A in the original string as the first occurrence of A in the first column (also labeled A₁). 

`$₁* * * * * A₁`

`A₁* * * * * N₁`

`A₂* * * * * N₂`

`A₃* * * * * B₁`

`B₁* * * * * $₁`

`N₁* * * * * A₂`

`N₂* * * * * A₃`


Remember that each row of our Burrows-Wheeler Matrix is simply a rotation of our original string. 

That means that, for any given row, in the original string, the letter in the first column comes immediately after the letter in the last column. 

For example, in the top row, A₁ is followed by $₁, and as we recall from our original string (BANANA$), the $ symbol did indeed appear after an A). We can use this "last-to-first" relationship to reconstruct our original string. We will depict the reconstruction of our original string in the top row.

<center><img src="https://sun9-53.userapi.com/impg/otmV2qsNe3d8cj0yy0hrgcHEMs-vgkvZqZdJWA/OuiT8n2wjYs.jpg?size=482x407&quality=96&proxy=1&sign=6055080098420318974024ed60c8654e&type=album" width=500/></center>

We have now reconstructed our original string in the top row! 

Most importantly, we didn't even need to reconstruct the entire Burrows-Wheeler Matrix: all we needed were the first and last columns! 

Thus, to reconstruct a string of length n from its Burrows-Wheeler Transform, we only needed **O(n) memory** (because both the first and last columns have exactly n rows), and after sorting the BWT to construct the first column, which can be done in **O(n) time** using radix sort if our alphabet is a constant size (e.g. ASCII), the actual reconstruction has **a worst-case time complexity of O(n)**.

### How does the BWT actually help us with fast pattern matching?

It turns out that we can slightly modify the logic we used in our reconstruction algorithm in order to find exact matches to our queries.

First, I need to keep track of a bit more information. 

Specifically, I also need to have some way of quickly mapping where any given letter in the last column appears in the first column. 

To do so, in addition to my first and last columns, I'll also keep track of a "last-to-first" column (labeled L2F in the table below):

* In Row 0, the character in the Last column is A₁, which appears in the First column in Row 1, so L2F in Row 0 is 1
* In Row 1, the character in the Last column is N₁, which appears in the First column in Row 5, so L2F in Row 1 is 5
* In Row 2, the character in the Last column is N₂, which appears in the First column in Row 6, so L2F in Row 2 is 6
* In Row 3, the character in the Last column is B₁, which appears in the First column in Row 4, so L2F in Row 3 is 4
* In Row 4, the character in the Last column is $₁, which appears in the First column in Row 0, so L2F in Row 4 is 0
* In Row 5, the character in the Last column is A₂, which appears in the First column in Row 2, so L2F in Row 2 is 2
* In Row 6, the character in the Last column is A₃, which appears in the First column in Row 3, so L2F in Row 3 is 3

<center><img src="https://sun9-31.userapi.com/impg/uOQ8ws7xmTXkTYoVypOviu31inNxRF37m3G0og/BvqggTXxzXY.jpg?size=503x350&quality=96&proxy=1&sign=d4e83e362a614e3e00ac6f9bee131758&type=album" width=500/></center>

### Pattern Matching

```
find(w):
    top = 0
    bottom = n-1
    for each character c in w in reverse order:
        if c does not exist in the range Last[top,bottom]:
            return null // w does not exist
        i = first occurrence of c in the range Last[top,bottom]
        j = last occurrence of c in the range Last[top,bottom]
        top = L2F[i]
        bottom = L2F[j]
    return all positions in original string corresponding to First[top,bottom]
```


<center><img src="https://sun9-31.userapi.com/impg/uOQ8ws7xmTXkTYoVypOviu31inNxRF37m3G0og/BvqggTXxzXY.jpg?size=503x350&quality=96&proxy=1&sign=d4e83e362a614e3e00ac6f9bee131758&type=album" width=500/></center>

Let's try to search for the query **AN**. 

We start with top as 0 and bottom as 6, and we iterate over the characters of our query in reverse order:

* The first letter we check is **N**. The first occurrence of N in $Last[0,6]$ is in Row 1 (N₁), and the last occurrence of N in $Last[0,6]$ is in Row 2 (N₂). Therefore, we set top to $L2F[1]$, which is 5, and we set bottom to $L2F[2]$, which is 6
* The next letter we check is **A**. The first occurrence of A in $Last[5,6]$ is in Row 5 (A₂), and the last occurrence of A in $Last[5,6]$ is in Row 6 (A₃). Therefore, we set top to $L2F[5]$, which is 2, and we set bottom to $L2F[6]$, which is 3
* We've finished iterating over all the letters of our query, so we're done! We ended with top = 2 and bottom = 3, so our query occurs in our original string starting at every letter in $First[2,3]$, i.e., wherever A₂ and A₃ occur in the original string

Let's try to search for the query **NAN**.

We start with top as 0 and bottom as 6, and we iterate over the characters of our query in reverse order:

* The first letter we check is N. The first occurrence of N in $Last[0,6]$ is in Row 1 (N₁), and the last occurrence of N in $Last[0,6]$ is in Row 2 (N₂). Therefore, we set top to $L2F[1]$, which is 5, and we set bottom to $L2F[2]$, which is 6
* The next letter we check is A. The first occurrence of A in $Last[5,6]$ is in Row 5 (A₂), and the last occurrence of A in $Last[5,6]$ is in Row 6 (A₃). Therefore, we set top to $L2F[5]$, which is 2, and we set bottom to $L2F[6]$, which is 3
* The next letter we check is N. The first occurrence of N in $Last[2,3]$ is in Row 2 (N₂), and the last occurrence of N in $Last[2,3]$ is also in Row 2 (N₂). Therefore, we set both top and bottom to $L2F[2]$, which is 6

We've finished iterating over all the letters of our query, so we're done! 

We ended with top = 6 and bottom = 6, so our query occurs in our original string starting at every letter in $First[6,6]$, i.e., wherever N₂ occurs in the original string

Let's try to search for the query **NAB**.

We start with top as 0 and bottom as 6, and we iterate over the characters of our query in reverse order:

* The first letter we check is **B**. The first occurrence of B in $Last[0,6]$ is in Row 3 (B₁), and the last occurrence of B in $Last[0,6]$ is also in Row 3 (B₁). Therefore, we set both top and bottom to $L2F[3]$, which is 4
* The next letter we check is A. The letter A does not occur at all in $Last[4,4]$, meaning we've failed! 

Our query does not exist in our original string
